In [1]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

CLASS_NAME = ['__background__', 'helmet', 'head', 'person']
DEVICE = torch.device('cpu')
# load Faster RCNN pre-trained model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# get the number of input features 
in_features = model.roi_heads.box_predictor.cls_score.in_features

# define a new head for the detector with required number of classes
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, len(CLASS_NAME)) 

checkpoint = torch.load('./savemodel/best_model.pth', map_location=DEVICE)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE).eval()


C:\Users\y.zhukov\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\y.zhukov\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [8]:


import torch 
import numpy as np
import  matplotlib.pyplot as plt
# any detection having score below this will be discarded, i.e bbox with scores more than 0.8
# will be shown, less than the threshold will not be shown
detection_threshold = 0.5

image = plt.imread('./image.png')
img = image.copy()

# bring color channels to front
img = np.transpose(img, (2, 0, 1)).astype(np.float32)

# convert to tensor; tensor([  3., 416., 416.], device='cuda:0')
img = torch.tensor(img, dtype=torch.float).cpu() #gpu enabled
    
# add batch dimension
img = torch.unsqueeze(img, 0)
with torch.no_grad(): #forward pass
    outputs = model(img.to(DEVICE))

# load all detection to CPU for further operations
outputs = [{k: v.to('cpu') for k, v in t.items()} for t in outputs]

# carry further only if there are detected boxes
if len(outputs[0]['boxes']) > 0:
    boxes = outputs[0]['boxes'].data.numpy() #converting tensor coordinates to numpy array
    scores = outputs[0]['scores'].data.numpy()
    lbls = outputs[0]['labels'].data.numpy()

    # filter out boxes according to `detection_threshold`
    boxes = boxes[scores > detection_threshold].astype(np.int32)
    lbls = lbls[scores > detection_threshold].astype(np.int32)
    pred_boxes = boxes.copy()
    # get all the predicited class names
    pred_class = [CLASS_NAME[i] for i in lbls]
    print(pred_class)


['helmet', 'helmet', 'helmet']
